In [1]:
import pandas as pd
import numpy as np
import itertools
import glob
import re
import jieba

After人工整理场景人群首购复购精神认同相关的聚类段落
需要先去重后再做进一步的精准修剪

In [2]:
# 去重的优先级，如果在negative和no_keyword_negative里都有，那么最后出现在no_keyword_negative里
senti_rank = {'no_keyword_negative':5, 
              'no_keyword_positive':4,
              'negative':2,
              'positive':1,
              'no_keyword_neutral':6,
              'neutral':3,
              'no_keyword':7
             }

In [3]:
def expand_row_paragraphs_to_columns_with_exist_rank(rebuy, index_columns_names, expand_column_name):
    """
        rebuy: 聚类段落数据
        index_columns_names: 标识段落属性的index, 比如id, cluster_size, center_sentences
        expand_column_name: 段落列，要把,拼接的段落变成一行一句话
    """
    index_columns = np.array(rebuy[index_columns_names])
    paragraphs = np.array(rebuy[expand_column_name])
    
    row_index_groups = []
    new_df = pd.DataFrame()
    for row_index, paragraph in zip(index_columns, paragraphs):
        row_index_groups.append(row_index)
        sentence_list = paragraph.split(',')
        paragraph_len = len(sentence_list)
        row_index = list(row_index)
        a = pd.concat([pd.DataFrame([row_index] *paragraph_len, columns = index_columns_names), 
                       pd.Series(sentence_list, name=expand_column_name)], axis=1)
        new_df = pd.concat([new_df, a])
    new_df = new_df.set_index(index_columns_names)
    return new_df, row_index_groups

In [39]:
def get_index_data(data, *args):
    return data.loc[pd.IndexSlice[args], :]

In [41]:
def revert_to_flat_cluster(new_df, row_index_groups):
    final_data = []
    for row_index in row_index_groups:
        try:
            row_index = list(row_index)
            para_list = get_index_data(new_df, *row_index)
            para = ",".join(para_list[column_to_clean].to_list())
            para_len = len(para_list)
            row_content = row_index + [para_len, para]
            final_data.append(row_content)
        except:
            print("这个类没了：")
            print(row_index)
    return final_data

In [376]:
# for sheet_name in ['首购复购', '人群', "适用场景", "精神认同_final"]:
for sheet_name in ['首购复购', '人群', "适用场景"]:

    rebuy = pd.read_excel(r"场景人群代言复购语料库\场景人群复购精神认同语料库2.xlsx", sheet_name = sheet_name)
    rebuy = rebuy[rebuy["tag_1"].notnull()]
    rebuy['tag_2'].fillna('TBD', inplace=True)
    rebuy.drop('段落', axis=1, inplace=True)
    
    rebuy = rebuy.reset_index()   # 记住原来的index，为了后面去完重按照原始的顺序输出
    index_columns_names = list(rebuy.columns[:5])
    column_to_clean = '段落_更新'
    
    rebuy = rebuy.sort_values(by='NLP', key=lambda x: x.replace(senti_rank))  # 根据自定义顺序去重
    
    new_df, row_index_groups = expand_row_paragraphs_to_columns_with_exist_rank(rebuy,
                                                                                index_columns_names, 
                                                                                expand_column_name=column_to_clean)
    
    new_df = new_df.drop_duplicates(subset = [column_to_clean], keep='first')
    final_data = revert_to_flat_cluster(new_df, row_index_groups)
    index_columns_names.extend(['评论数_去重','去重段落'])
    result = pd.DataFrame(final_data, columns=index_columns_names).sort_values(by='index')
    
    result.to_excel(f"{sheet_name}.xlsx",index=False)

In [379]:
def expand(df):
    return pd.DataFrame({"段落_去重": itertools.chain(*df["段落_更新"].str.split(","))})
# rebuy.groupby(['tag_1', 'tag_2', '评论数', 'NLP']).apply(expand)

### 把所有聚类结果合并去重

In [80]:
def expand_column(df):
    return pd.DataFrame({"去重段落": itertools.chain(*df['cluster_sentences'].str.split(','))})

def fold_back(df):
    return ",".join(list(df.to_list()))

In [78]:
# 拼接所有情感类别的聚类结果
file_list = glob.glob(r"model_result\cluster\classify_sample_1_*_ascending_*.xlsx")
combine_cluster_result = pd.DataFrame()
for file_name in file_list:
    sentiment_type = re.match(r'.+classify_sample_1_(.+)_cos_kmeans', file_name).group(1)
    if sentiment_type == 'no_keyword':   # 最早keyword没有按照三种情感分别聚类，不分情感的现在不需要了（已检查过没有什么价值）
        continue
    data = pd.read_excel(file_name)
    data = pd.concat([pd.Series([sentiment_type]*len(data), name='sentiment'), data], axis=1)
    combine_cluster_result = pd.concat([combine_cluster_result, data])

In [ ]:
# 按照自定义优先级对段落句子去重
grp_index = ['sentiment', 'cluster_id', 'cluster_size','center_sentence']
column_to_clean = 'cluster_sentences'

combine_cluster_result = combine_cluster_result.sort_values(by='sentiment', key=lambda x: x.replace(senti_rank))
new_df, row_index_groups = expand_row_paragraphs_to_columns_with_exist_rank(combine_cluster_result,
                                                                            grp_index, 
                                                                            expand_column_name=column_to_clean)

new_df = new_df.drop_duplicates(subset = [column_to_clean], keep='first')
final_data = revert_to_flat_cluster(new_df, row_index_groups)
grp_index.extend(["去重段落数", "去重段落"])
combine_cluster_no_dup = pd.DataFrame(final_data, columns=grp_index)
combine_cluster_no_dup.to_excel("step_data/cluster_result_combine_all_sentiments_no_dup_update.xlsx", index=False)

In [261]:
# 方法二
grp_index = ['sentiment', 'cluster_id', 'cluster_size','center_sentence']
# 把每个类里面由,拼接的多个句子扩展变成一列，每行一句话
combine_cluster_result = combine_cluster_result.groupby(grp_index).apply(expand_column).reset_index()

#按照senti_rank自定义优先级去重
combine_cluster_result = combine_cluster_result.sort_values(by=['sentiment','level_4'], key=lambda x: x.replace(senti_rank))
# combine_cluster_result = combine_cluster_result.sort_values(by=['sentiment'], key=lambda x: x.replace(senti_rank)) #坑版

remove_duplicate = combine_cluster_result.drop_duplicates(subset=['去重段落'], keep='first')

# 把每个类的句子再逗号拼接回去
columns_name = grp_index + ["去重段落数", "去重段落"]
combine_cluster_no_dup = remove_duplicate.groupby(grp_index).agg({'去重段落':['count', fold_back]}).reset_index()
combine_cluster_no_dup.columns = columns_name
combine_cluster_no_dup = combine_cluster_no_dup.sort_values(by='sentiment', key=lambda x: x.replace(senti_rank))
combine_cluster_no_dup.to_excel("step_data/cluster_result_combine_all_sentiments_no_dup.xlsx", index=False)

### 如果是已经拿出来的人群，适用场景，精神认同，首购复购，则先打上tag_1

In [44]:
combine_cluster_no_dup = pd.read_excel("step_data/cluster_result_combine_all_sentiments_no_dup_update.xlsx")

In [46]:
sheet_name='首购复购'
tagged_cluster = pd.DataFrame()
for sheet_name in ['首购复购', '人群', "适用场景", "精神认同"]:
    data = pd.read_excel(r"场景人群代言复购语料库\场景人群复购精神认同语料库_keysentence.xlsx", sheet_name = sheet_name)
    data = data[data["tag_1"].notnull()]
    data['中心语句'] = data['中心语句'].fillna(pd.Series(data["段落"].str.split(',')[0]))
    tagged_cluster = pd.concat([tagged_cluster, data[['中心语句','tag_1','NLP']]])
    
tagged_cluster.columns = ['center_sentence', 'tag_1', 'sentiment']
tagged_cluster = tagged_cluster.drop_duplicates()
cluster_tag = pd.merge(combine_cluster_no_dup, tagged_cluster, on=['sentiment','center_sentence'], how='left')

#### 其余的根据中心语句的关键字和关键词词表打标签

In [51]:
jieba.load_userdict(r"model_input/keywords/add_keywords_for_jieba.txt")

In [53]:
# 读取关键词
keyword_tag=pd.read_excel(r'model_input\keywords\keywords_sentimental_words.xlsx')
keyword_tag=keyword_tag.loc[:,['关键字','一级','二级','情感倾向']]

In [54]:
def find_keyword(s):
    for word in s:
        if word in keyword_tag['关键字'].to_list():
            return word
    return None
cluster_tag['中心语断句'] = cluster_tag['center_sentence'].apply(lambda x: list(jieba.cut(x, cut_all=True)))
cluster_tag['关键字'] = cluster_tag['中心语断句'].apply(find_keyword)

In [55]:
cluster_tag = pd.merge(cluster_tag, keyword_tag, on='关键字',how='left')
cluster_tag['tag_1'] = cluster_tag['tag_1'].fillna(cluster_tag['一级'])

In [56]:
cluster_tag.to_excel("step_data/cluster_result_combine_all_sentiments_no_dup_tag.xlsx", index=False)

In [64]:
cluster_tag.shape

(874, 12)

In [76]:
old = pd.read_excel("step_data/cluster_result_combine_all_sentiments_no_dup_tag3.xlsx")
old = old[["sentiment", 'cluster_id', "center_sentence","一级", "二级", "Doris", '处理级别', '备注']]
cluster_tag.drop(['一级','二级','情感倾向'], axis=1, inplace=True)
cluster_tag2 = pd.merge(cluster_tag, old, how='left', on=['sentiment', 'cluster_id', 'center_sentence'])
cluster_tag2.to_excel("step_data/cluster_result_combine_all_sentiments_no_dup_tag4.xlsx", index=False)

In [77]:
cluster_tag2.groupby("center_sentence")['sentiment'].count().sort_values(ascending=False)[:10]

center_sentence
给老爸跑步买的                     4
又给我媳妇买了一双                   4
舒适度跑步很舒服啊                   2
碳板的弹性调教的不像乔丹飞影PBnext那么突兀    2
一直穿特步的鞋子质量很穿上还舒服            2
舒适度爱了                       1
舒适度很不错的                     1
舒适度很佳                       1
舒适度很好啦                      1
舒适度很舒服哟                     1
Name: sentiment, dtype: int64

In [85]:
cluster_tag["去重段落"][98:99].to_list()

['风格款式介绍配色挺好看,风格款式介绍配色挺好看的,风格款式介绍配色很好看啊,风格款式介绍配色也很好看,风格款式介绍颜色挺好看的,风格款式介绍配色很漂亮,风格款式介绍颜色配色很好看,风格款式介绍颜色挺好看,风格款式介绍挺好搭配,风格款式介绍配色真的很好看,风格款式介绍外形挺好看的,风格款式介绍新款挺好看的,风格款式介绍新款挺好看,风格款式介绍颜色搭配很不错,风格款式介绍版型挺好看的,风格款式介绍好看配色超棒啊,风格款式介绍款式也挺好看,风格款式介绍挺漂亮,风格款式介绍样子挺好看,风格款式介绍红色挺漂亮的,风格款式介绍样子挺好看的,风格款式介绍颜色搭配不错,风格款式介绍颜色很好看,风格款式介绍颜色也很好看,风格款式介绍挺好看这个色,风格款式介绍颜值很好看,风格款式介绍颜色还挺耐看的,风格款式介绍白蓝款挺好看,风格款式介绍蓝色还挺好看的,风格款式介绍配色超级好看,风格款式介绍白的挺好看的,风格款式介绍款式还挺好看,风格款式介绍颜色搭配很耐看,风格款式介绍颜色很好看呀,风格款式介绍颜色很百搭,风格款式介绍挺漂亮的,风格款式介绍粉色挺好看哒,风格款式介绍款式还挺好看的,风格款式介绍还挺好搭配的,风格款式介绍银白外观挺好看,风格款式介绍颜值挺高的,风格款式介绍款式还是挺好看,风格款式介绍颜值可以挺好看的,风格款式介绍这个款色挺好看,风格款式介绍样子很好看,风格款式介绍样子是挺好看的,风格款式介绍色彩搭配不错,风格款式介绍全黑色的很好看,风格款式介绍这个款式挺好看,风格款式介绍灰色的挺耐看的,风格款式介绍侧面很好看,风格款式介绍样式挺漂亮的,风格款式介绍外形也很好看,风格款式介绍款式挺漂亮的,风格款式介绍样子也挺好看,风格款式介绍款式搭配也很不错,风格款式介绍颜色挺好,风格款式介绍象牙色的挺好看的,风格款式介绍颜色也不错,风格款式介绍样式挺时尚的,风格款式介绍颜色非常好看,风格款式介绍这个色彩搭配很好看,风格款式介绍新款很好看,风格款式介绍外形很好看,风格款式介绍外观挺好看的,风格款式介绍颜色整体很好看,风格款式介绍颜色搭配非常漂亮,风格款式介绍新款挺漂亮,风格款式介绍款式还是挺好看的,风格款式介绍款式很好搭配,风格款式介绍挺时尚的,风格款式介绍风格还是挺好看的,风格款式介绍整体搭配非常好,风格款式介绍挺好看的颜色,风格款式介绍挺秀气的,风格款式介绍挺好看诶,风格款式介绍款式

In [271]:
cluster_tag = pd.read_excel("step_data/cluster_result_combine_all_sentiments_no_dup_tag2.xlsx")

In [220]:
def top_words(s):
    sentences_list = s.split(",")
    words_list = [list(jieba.cut(sentence)) for sentence in sentences_list]
    words_list =list(itertools.chain(*words_list))
    words_list = [word for word in words_list if word not in stopwords]
    topwords = FreqDist(words_list)
    top_words = topwords.most_common(len(sentences_list)*0.)
cluster_tag["去重段落"].apply()


True

In [230]:
from nltk.probability import FreqDist

In [231]:
topwords = FreqDist(["a",'b','c','c'])
topwords.most_common(2)

[('c', 2), ('a', 1)]

In [245]:
def read_list(path):
    words = [line.strip() for line in open(path, encoding='UTF-8').readlines()]
    words = np.unique(words, axis=0) # 删除重复数据
    return words
stopwords_path=r'model_input/keywords/cn_stopwords.txt'
stopwords=read_list(stopwords_path)

In [254]:
s = cluster_tag.loc[38, '去重段落']
sentences_list = s.split(",")
words_list = [list(jieba.cut(sentence)) for sentence in sentences_list]
words_list =list(itertools.chain(*words_list))
words_list = [word for word in words_list if word not in stopwords]
topwords = FreqDist(words_list)
top_words = topwords.most_common(5)
top_words[-1][1] / len(sentences_list)

0.22910216718266255

In [ ]:
只对有一级标签的处理
处理级别列=0,1可以直接用，不需要再额外处理---0不用再处理1手工剔除2需要搜索处理同一级标签 3非同一级标签混搭4丢弃
人群跑步等tag的Doris处理我不处理, 如果一级标签是适用场景且处理级别是3，我要管，意味着段落里除了有运动场景，还有性能或其他一级标签
处理级别是3的可以先去搜索doris第一步出来的标签，没有的再加词搜索，

In [330]:
## 找句子
a = pd.read_excel(r"from Doris\聚类标签\negative_cluster_freq.xlsx")
"给丫头跑步穿应该会舒服" in list(itertools.chain(*(a['cluster_sentences'].str.split(","))))

True